## Import Required Dependencies

In [469]:
!pip install --upgrade pip
!pip install htmldate
!pip install xlsxwriter

In [635]:
import pandas as pd
import numpy as np
import re
from htmldate import find_date

## Required Functions

In [636]:
def sentiment_analysis(texto):
        analysis = TextBlob(texto)
        return analysis.sentiment.polarity
    
def crear_dicc_keywords(df_keywords):
    df_keywords=df_keywords.fillna('exxxtract')
    #df_keywords
    area_dict = df_keywords.to_dict('list')
    #print(area_dict)
    for k,v in area_dict.items():
        #print(v)
        nv=list(set(v))
        nv=[x for x in v if x != 'exxxtract']
        nv=list(set(nv))
        area_dict[k]=nv
    return area_dict
    
def contar_palabras(comentario, palabras):
    contador = 0
    for palabra in palabras:
        contador += len(re.findall(fr"\b{palabra}\b", comentario, re.IGNORECASE))
    return contador

def palabras_presentes(comentario, palabras):
    return [palabra for palabra in palabras if re.search(fr"\b{palabra}\b", comentario, re.IGNORECASE)]

## Read Categories and Topics

## Categories

In [637]:
categories = pd.read_excel('categories_topics/categorias_keywords__temas_amgen.xlsx', sheet_name= 'categorias')
categories = categories.drop(categories.filter(like='Unnamed')).copy()
print(categories.shape)
categories.head()

(73, 5)


,cat_Biocomparables,cat_Cardiovascular,cat_Corporativo,cat_Hematología,cat_Osteoporosis
0,#AmgenBiocomparables,cardiovascular,#WeAreAmgen,Mieloma Múltiple,Fundación Internacional de Osteoporosis
1,biocomparables,salud cardiovascular,#GreatPlaceToWork,neoplasias malignas,International Osteoporosis Foundation
2,Biocomparables,#AmgenCardiovascular,#SomosGPTW,neoplasias hematológicas,Osteoporosis
3,NaN,#ElColesterolNoAvisa,#LosMejores2023,mieloma múltiple,osteoporosis
4,NaN,#DíaMundialdelCorazón,#BeGreat,padecimientos,fracturas vertebrales


## Topics

In [638]:
topics = pd.read_excel('categories_topics/categorias_keywords__temas_amgen.xlsx', sheet_name='temas')
topics.columns = topics.columns.str.replace(' ', '_')
topics = topics.drop(topics.filter(like='Unnamed')).copy()

print(topics.shape)
topics.head()

(14, 20)


,topic_Alerta_Colesterol,topic_Biocomparables,topic_Cánceres_hematológicos,topic_Colesterol_LDL,topic_Día_de_la_independencia,topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva,topic_Día_Mundial_de_la_Leucemia,topic_Día_Mundial_de_la_seguridad_del_paciente,topic_Día_Mundial_de_la_Trombocitopenia_Inmune,topic_Día_Mundial_del_Corazón,topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos,topic_Estudio_enfermedades_cardiovasculares,topic_GPTW,topic_Hematopedia,topic_LDL_Xchange,topic_LLA,topic_Mieloma_Múltiple,topic_Reporte_ESG,topic_Salud_ósea,topic_We_Are_Amgen
0,colesterol,#AmgenBiocomparables,cánceres hematológicos,colesterol LDL,Independencia de México,discapacidad auditiva,Día Mundial de la Leucemia,Día Mundial de la Seguridad del Paciente,Día Mundial de la Trombocitopenia Inmune Primaria,#DíaMundialdelCorazón,#DonarSalvaVidas,enfermedades cardiovasculares,#WeAreAmgen,#SeptiembreHematológico,#LDLXChange,#LLA,#MM,#ESG2022,#AmgenSaludÓsea,#WeAreAmgen
1,HDL,biocomparables,#SeptiembreHematológico,#Redúcelo,¡Viva México!,NaN,Mundial,Seguridad,TIP,DíaMundialdelCorazón,DonarSalvaVidas,#AmgenCardiovascular,#GreatPlaceToWork,#LLA,#LDLXchange 2023,LLA,#MielomaMúltiple,ESG,International Osteoporosis Foundation,#deCodeGenetics
2,LDL,Biocomparables,#MM,#DíaMundialDelCorazón,Independencia,NaN,Leucemia,Paciente,Trombocitopenia,Corazón,NaN,NaN,#SomosGPTW,#Hematopedia,LDLXChange,#AmgenHematología,Mieloma Múltiple,trabajo ESG,Fundación Internacional de Osteoporosis,#AmgenScience
3,bueno,AmgenBiocomparables,#TIP,#WHD,NaN,NaN,NaN,NaN,Inmune,Corazon,NaN,NaN,#LosMejores2023,#TIP,LDLXchange 2023,#SeptiembreHematológico,mieloma múltiple,ESG2022,fracturas vertebrales,WeAreAmgen
4,malo,NaN,NaN,#ElColesterolNoAvisa,NaN,NaN,NaN,NaN,Primaria,corazon,NaN,NaN,#BeGreat,#MM,NaN,NaN,#DíaMundialDelMielomaMúltple,NaN,NaN,deCodeGenetics


## Read Source Raw Data

### Facebook

In [639]:
amgen_fb_df = pd.read_csv('source_data_raw/september/facebook/Sep-01-2023_Sep-30-2023_681592587250890.csv')
print(amgen_fb_df.shape)
amgen_fb_df.head(3)

(39, 37)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Duración (segundos),Hora de publicación,Tipo de subtítulo,Enlace permanente,Es una publicación cruzada,...,Total de clics,Tiempo promedio dedicado al artículo,Profundidad de desplazamiento diaria en el artículo,Clics de otro tipo,Visualizaciones de fotos,Clics en el enlace,Clics para reproducir,Impresiones del anuncio,CPM del anuncio (USD),Ingresos estimados (USD)
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,0,09/02/2023 12:09,NaN,https://www.facebook.com/AmgenMx/posts/pfbid02...,0,...,3,0,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,0,09/01/2023 08:09,NaN,https://www.facebook.com/AmgenMx/posts/pfbid0B...,0,...,1,0,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,0,09/30/2023 09:09,NaN,https://www.facebook.com/AmgenMx/posts/pfbid0A...,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [640]:
amgen_fb_df = amgen_fb_df[['Identificador de la publicación',
                              'Identificador de la página',
                              'Nombre de la página',
                              'Título',
                              'Descripción',
                              'Hora de publicación',
                              'Enlace permanente',
                              'Personas alcanzadas',
                              'Impresiones',
                              'Veces que se compartió',
                              'Me gusta',
                              'Comentarios',
                              'Total de clics',
                              'Reproducciones de video de 3 segundos']]
print(amgen_fb_df.shape)
amgen_fb_df.head(3)

(39, 14)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,Me gusta,Comentarios,Total de clics,Reproducciones de video de 3 segundos
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,13,0,3,0
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,5,0,1,0
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,3,0,0,0


In [641]:
amgen_fb_df['Interacciones']= amgen_fb_df['Veces que se compartió'] + amgen_fb_df['Me gusta'] + amgen_fb_df['Comentarios'] + amgen_fb_df['Total de clics']
print(amgen_fb_df.shape)
amgen_fb_df.head(3)

(39, 15)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,Me gusta,Comentarios,Total de clics,Reproducciones de video de 3 segundos,Interacciones
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,13,0,3,0,16
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,5,0,1,0,6
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,3,0,0,0,3


In [642]:
amgen_fb_df['Categoría'] = ''
amgen_fb_df['Tema'] = ''
amgen_fb_df['Formato'] = ''
amgen_fb_df['Día'] = ''
amgen_fb_df['Hora'] = ''
print(amgen_fb_df.shape)
amgen_fb_df.head(3)

(39, 20)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,Me gusta,Comentarios,Total de clics,Reproducciones de video de 3 segundos,Interacciones,Categoría,Tema,Formato,Día,Hora
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,13,0,3,0,16,,,,,
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,5,0,1,0,6,,,,,
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,3,0,0,0,3,,,,,


## Processing Categories

In [643]:
categories_dicc= crear_dicc_keywords(categories)
categories_dicc

{'cat_Biocomparables': ['biocomparables',
  '#AmgenBiocomparables',
  'Biocomparables'],
 'cat_Cardiovascular': ['salud cardiovascular',
  'acumulación de grasas',
  'colesterol',
  '#Redúcelo',
  '#ElColesterolNoAvisa',
  'cardiovasculares',
  'colesterol malo',
  'colesterol HDL',
  'colesterol malo (LDL)',
  'colesterol LDL',
  'ECV',
  'Sociedad Europea de Cardiología (ESC)',
  'PCSK9',
  'Dieta equilibrada',
  'niveles de colesterol',
  'colesterol bueno',
  '#WHD',
  '#DíaMundialDelCorazón',
  'ECA',
  '#AmgenCardiovascular',
  'dieta equilibrada',
  'arterias',
  '#LDLXChange',
  'accidentes cerebrovasculares',
  'alimentacion',
  '#AimLow',
  'Control del peso',
  'Día Mundial del Corazón',
  'peso saludable',
  'ejercicio regular',
  'corazón',
  'Organización Mundial de la Salud',
  'eventos cardiovasculares',
  'visitas regulares al medico',
  'terapia antilipídica',
  '#LDLXchange 2023',
  'enfermedad cardiovascular aterosclerótica (ECA)',
  'estilo de vida',
  'LDL',
  'in

In [644]:
for k,v in categories_dicc.items():
    amgen_fb_df[f'{k}']= amgen_fb_df['Descripción'].apply(lambda x: palabras_presentes(x, v))
    # in orher column count the finded words
    amgen_fb_df[f'count_{k}'] = amgen_fb_df['Descripción'].apply(lambda x: contar_palabras(x, v))
print(amgen_fb_df.shape)
amgen_fb_df.head()

(39, 30)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,cat_Biocomparables,count_cat_Biocomparables,cat_Cardiovascular,count_cat_Cardiovascular,cat_Corporativo,count_cat_Corporativo,cat_Hematología,count_cat_Hematología,cat_Osteoporosis,count_cat_Osteoporosis
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,[],0,[],0,"[Great Place to Work, Mejores Lugares para Tra...",3,[],0,[],0
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,[],0,"[salud cardiovascular, cardiovascular]",2,[Amgen],1,[],0,[],0
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,[],0,[Organización Mundial de la Salud],1,[Amgen],1,"[Mieloma Múltiple, neoplasias hematológicas, p...",11,[],0
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,[],0,"[colesterol, cardiovasculares, colesterol malo...",9,[Amgen],1,[],0,[],0
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,[],0,[],0,[Amgen],1,"[plaquetas, TIP, trombocitopenia, Día Mundial ...",6,[],0


In [645]:
def assign_categoria(count_cat_Cardiovascular, count_cat_Hematología, count_cat_Osteoporosis, count_cat_Biocomparables, count_cat_Corporativo):
    if any([count_cat_Cardiovascular > 1, count_cat_Hematología > 1, count_cat_Osteoporosis > 1, count_cat_Biocomparables > 1, count_cat_Corporativo > 1]):
        categoria = 'Cardiovascular' if count_cat_Cardiovascular > 1 else 'Hematología' if count_cat_Hematología > 1 else 'Osteoporosis' if count_cat_Osteoporosis > 1 else 'Biocomparables' if count_cat_Biocomparables > 1 else 'Corporativo'
    else:
        categoria = None
    return categoria

In [646]:
amgen_fb_df['Categoría_pred'] = amgen_fb_df.apply(lambda row: assign_categoria(row['count_cat_Cardiovascular'], row['count_cat_Hematología'], row['count_cat_Osteoporosis'], row['count_cat_Biocomparables'], row['count_cat_Corporativo']), axis=1)
print(amgen_fb_df.shape)
amgen_fb_df.head()

(39, 31)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,count_cat_Biocomparables,cat_Cardiovascular,count_cat_Cardiovascular,cat_Corporativo,count_cat_Corporativo,cat_Hematología,count_cat_Hematología,cat_Osteoporosis,count_cat_Osteoporosis,Categoría_pred
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,0,[],0,"[Great Place to Work, Mejores Lugares para Tra...",3,[],0,[],0,Corporativo
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,0,"[salud cardiovascular, cardiovascular]",2,[Amgen],1,[],0,[],0,Cardiovascular
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,0,[Organización Mundial de la Salud],1,[Amgen],1,"[Mieloma Múltiple, neoplasias hematológicas, p...",11,[],0,Hematología
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,0,"[colesterol, cardiovasculares, colesterol malo...",9,[Amgen],1,[],0,[],0,Cardiovascular
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,0,[],0,[Amgen],1,"[plaquetas, TIP, trombocitopenia, Día Mundial ...",6,[],0,Hematología


In [647]:
amgen_fb_df['Categoría_pred']

0        Corporativo
1     Cardiovascular
2        Hematología
3     Cardiovascular
4        Hematología
5     Cardiovascular
6        Corporativo
7     Cardiovascular
8               None
9     Cardiovascular
10    Cardiovascular
11       Hematología
12    Cardiovascular
13    Cardiovascular
14    Cardiovascular
15       Hematología
16       Hematología
17       Hematología
18    Cardiovascular
19       Hematología
20    Cardiovascular
21       Hematología
22    Cardiovascular
23    Cardiovascular
24              None
25    Cardiovascular
26    Biocomparables
27    Cardiovascular
28       Corporativo
29    Cardiovascular
30      Osteoporosis
31       Hematología
32    Cardiovascular
33    Cardiovascular
34       Corporativo
35       Hematología
36       Hematología
37       Hematología
38              None
Name: Categoría_pred, dtype: object

## Processing Topics

In [648]:
topics_dicc= crear_dicc_keywords(topics)
topics_dicc

{'topic_Alerta_Colesterol': ['HDL',
  'colesterol',
  'LDL',
  'malo',
  '#AmgenCardiovascular',
  '#ElColesterolNoAvisa',
  'Marco Antonio Alcocer Gamba',
  'bueno'],
 'topic_Biocomparables': ['AmgenBiocomparables',
  'biocomparables',
  '#AmgenBiocomparables',
  'Biocomparables'],
 'topic_Cánceres_hematológicos': ['#SeptiembreHematológico',
  '#TIP',
  'cánceres hematológicos',
  '#MM'],
 'topic_Colesterol_LDL': ['#Redúcelo',
  '#ElColesterolNoAvisa',
  '#AmgenCardiovascular',
  'colesterol malo',
  'colesterol malo (LDL)',
  'colesterol LDL',
  '#AimLow',
  '#WHD',
  '#DíaMundialDelCorazón'],
 'topic_Día_de_la_independencia': ['Independencia',
  'Independencia de México',
  '¡Viva México! '],
 'topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva': ['discapacidad auditiva'],
 'topic_Día_Mundial_de_la_Leucemia': ['Mundial',
  'Leucemia',
  'Día Mundial de la Leucemia'],
 'topic_Día_Mundial_de_la_seguridad_del_paciente': ['Día Mundial de la Seguridad del Paciente',
  'Pac

In [649]:
for k,v in topics_dicc.items():
    amgen_fb_df[f'{k}']= amgen_fb_df['Descripción'].apply(lambda x: palabras_presentes(x, v))
    # in orher column count the finded words
    amgen_fb_df[f'count_{k}'] = amgen_fb_df['Descripción'].apply(lambda x: contar_palabras(x, v))
print(amgen_fb_df.shape)
amgen_fb_df.head()

(39, 71)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,topic_LLA,count_topic_LLA,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,[],0,[],0,[],0,[],0,[WeAreAmgen],1
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,[],0,[],0,[],0,[],0,[],0
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,[],0,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,[],0,[],0,[],0,[],0,[],0
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,[],0,[],0,[],0,[],0,[],0


In [650]:
print(amgen_fb_df.columns)

Index(['Identificador de la publicación', 'Identificador de la página',
       'Nombre de la página', 'Título', 'Descripción', 'Hora de publicación',
       'Enlace permanente', 'Personas alcanzadas', 'Impresiones',
       'Veces que se compartió', 'Me gusta', 'Comentarios', 'Total de clics',
       'Reproducciones de video de 3 segundos', 'Interacciones', 'Categoría',
       'Tema', 'Formato', 'Día', 'Hora', 'cat_Biocomparables',
       'count_cat_Biocomparables', 'cat_Cardiovascular',
       'count_cat_Cardiovascular', 'cat_Corporativo', 'count_cat_Corporativo',
       'cat_Hematología', 'count_cat_Hematología', 'cat_Osteoporosis',
       'count_cat_Osteoporosis', 'Categoría_pred', 'topic_Alerta_Colesterol',
       'count_topic_Alerta_Colesterol', 'topic_Biocomparables',
       'count_topic_Biocomparables', 'topic_Cánceres_hematológicos',
       'count_topic_Cánceres_hematológicos', 'topic_Colesterol_LDL',
       'count_topic_Colesterol_LDL', 'topic_Día_de_la_independencia',
       '

In [651]:
amgen_fb_df['topic_LDL_Xchange']

0                                []
1     [LDLXChange, LDLXchange 2023]
2                                []
3                                []
4                                []
5                                []
6                                []
7                                []
8                                []
9                                []
10                               []
11                               []
12    [LDLXChange, LDLXchange 2023]
13                               []
14                               []
15                               []
16                               []
17                               []
18                               []
19                               []
20                               []
21                               []
22                     [LDLXChange]
23                               []
24                               []
25                               []
26                               []
27                          

In [652]:
def assign_topico(count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Cánceres_hematológicos, count_topic_Colesterol_LDL, count_topic_Día_de_la_independencia, count_topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Día_Mundial_de_la_Leucemia, count_topic_Día_Mundial_de_la_seguridad_del_paciente, count_topic_Día_Mundial_de_la_Trombocitopenia_Inmune, count_topic_Día_Mundial_del_Corazón, count_topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Múltiple, count_topic_Reporte_ESG, count_topic_Salud_ósea, count_topic_We_Are_Amgen):
    if any([count >= 1 for count in [count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Cánceres_hematológicos, count_topic_Colesterol_LDL, count_topic_Día_de_la_independencia, count_topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Día_Mundial_de_la_Leucemia, count_topic_Día_Mundial_de_la_seguridad_del_paciente, count_topic_Día_Mundial_de_la_Trombocitopenia_Inmune, count_topic_Día_Mundial_del_Corazón, count_topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Múltiple, count_topic_Reporte_ESG, count_topic_Salud_ósea,count_topic_We_Are_Amgen]]):
        topico = next(count_name for count_name, count in zip(["Alerta Colesterol", "Biocomparables", "Cánceres hematológicos", "Colesterol LDL", "Día de la independencia", "Día Internacional de las personas con discapacidad auditiva", "Día Mundial de la Leucemia", "Día Mundial de la seguridad del paciente", "Día Mundial de la Trombocitopenia Inmune", "Día Mundial del Corazón", "Día Nacional de la donación y trasplante de órganos", "Estudio enfermedades cardiovasculares", "GPTW", "Hematopedia", "LDL Xchange", "LLA", "Mieloma Múltiple", "Reporte ESG", "Salud Ósea", "We Are Amgen"], [count_topic_Alerta_Colesterol,count_topic_Biocomparables,count_topic_Cánceres_hematológicos,count_topic_Colesterol_LDL,count_topic_Día_de_la_independencia,count_topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva,count_topic_Día_Mundial_de_la_Leucemia,count_topic_Día_Mundial_de_la_seguridad_del_paciente,count_topic_Día_Mundial_de_la_Trombocitopenia_Inmune,count_topic_Día_Mundial_del_Corazón,count_topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos,count_topic_Estudio_enfermedades_cardiovasculares,count_topic_GPTW,count_topic_Hematopedia,count_topic_LDL_Xchange,count_topic_LLA,count_topic_Mieloma_Múltiple,count_topic_Reporte_ESG,count_topic_Salud_ósea,count_topic_We_Are_Amgen]) if count >= 1)
    else:
        topico = None
    return topico

In [653]:
#def assign_topico(count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, count_topic_Dia_Mundial_del_Corazon, count_topic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Multiple, count_topic_Reporte_ESG, count_topic_Salud_osea, count_topic_We_Are_Amgen):
#    if any(count > 1 for count in [count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, count_topic_Dia_Mundial_del_Corazon, count_topic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Multiple, count_topic_Reporte_ESG, count_topic_Salud_osea, count_topic_We_Are_Amgen]):
#        topico = next(count_name for count_name, count in zip(["Alerta Colesterol", "Biocomparables", "Canceres hematológicos", "Colesterol LDL", "Día de la independencia", "Día Internacional de las personas con discapacidad auditiva", "Día Mundial de la Leucemia", "Día Mundial de la seguridad del paciente", "Día Mundial de la Trombocitopenia Inmune", "Día Mundial del Corazón", "Día Nacional de la donación y trasplante de órganos", "Estudio enfermedades cardiovasculares", "GPTW", "Hematopedia", "LDL Xchange", "LLA", "Mieloma Múltiple", "Reporte ESG", "Salud Ósea", "We Are Amgen"], [count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, count_topic_Dia_Mundial_del_Corazon, count_topic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Multiple, count_topic_Reporte_ESG, count_topic_Salud_osea, count_topic_We_Are_Amgen]) if count > 1)
#    else:
#        topico = None
#    return topico

In [654]:
#def assign_topico(count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, count_topic_Dia_Mundial_del_Corazon, count_topic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Multiple, count_topic_Reporte_ESG, count_topic_Salud_osea, count_topic_We_Are_Amgen):
#    if any(count > 1 for count in [count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, count_topic_Dia_Mundial_del_Corazon, count_topic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos, count_topic_Estudio_enfermedades_cardiovasculares, count_topic_GPTW, count_topic_Hematopedia, count_topic_LDL_Xchange, count_topic_LLA, count_topic_Mieloma_Multiple, count_topic_Reporte_ESG, count_topic_Salud_osea, count_topic_We_Are_Amgen]):
#        topico = next(count_name for count_name, count in zip(["Alerta Colesterol", "Biocomparables", "Canceres hematológicos", "Colesterol LDL", "Día de la independencia", "Día Internacional de las personas con discapacidad auditiva", "Día Mundial de la Leucemia", "Día Mundial de la seguridad del paciente", "Día Mundial de la Trombocitopenia Inmune", "Día Mundial del Corazón", "Día Nacional de la donación y trasplante de órganos", "Estudio enfermedades cardiovasculares", "GPTW", "Hematopedia", "LDL Xchange", "LLA", "Mieloma Múltiple", "Reporte ESG", "Salud Ósea", "We Are Amgen"], [count_topic_Alerta_Colesterol, count_topic_Biocomparables, count_topic_Canceres_hematologicos, count_topic_Colesterol_LDL, count_topic_Dia_de_la_independencia, count_topic_Dia_Internacional_de_las_personas_con_discapacidad_auditiva, count_topic_Dia_Mundial_de_la_Leucemia, count_topic_Dia_Mundial_de_la_seguridad_del_paciente, count_topic_Dia_Mundial_de_la_Trombosis_Inmune, ountopic_Dia_Mundial_del_Corazon,countopic_Dia_Nacional_de_la_donacion_y_trasplante_de_organos,countopic_Estudio_enfermedades_cardiovasculares,countopic_GPTW,countopic_Hematopedia,countopic_LDL_Xchange,countopic_LLA,countopic_Mieloma_Multiple,countopic_Reporte_ESG,countopic_Salud_osea,countopic_We_Are_Amgen]) if count > 1)
#    else:
#        topico = None
#    return topico

In [655]:
print(amgen_fb_df.columns)

Index(['Identificador de la publicación', 'Identificador de la página',
       'Nombre de la página', 'Título', 'Descripción', 'Hora de publicación',
       'Enlace permanente', 'Personas alcanzadas', 'Impresiones',
       'Veces que se compartió', 'Me gusta', 'Comentarios', 'Total de clics',
       'Reproducciones de video de 3 segundos', 'Interacciones', 'Categoría',
       'Tema', 'Formato', 'Día', 'Hora', 'cat_Biocomparables',
       'count_cat_Biocomparables', 'cat_Cardiovascular',
       'count_cat_Cardiovascular', 'cat_Corporativo', 'count_cat_Corporativo',
       'cat_Hematología', 'count_cat_Hematología', 'cat_Osteoporosis',
       'count_cat_Osteoporosis', 'Categoría_pred', 'topic_Alerta_Colesterol',
       'count_topic_Alerta_Colesterol', 'topic_Biocomparables',
       'count_topic_Biocomparables', 'topic_Cánceres_hematológicos',
       'count_topic_Cánceres_hematológicos', 'topic_Colesterol_LDL',
       'count_topic_Colesterol_LDL', 'topic_Día_de_la_independencia',
       '

In [656]:
amgen_fb_df['Tema_pred'] = amgen_fb_df.apply(lambda row: assign_topico(row['count_topic_Alerta_Colesterol'], row['count_topic_Biocomparables'], row['count_topic_Cánceres_hematológicos'], row['count_topic_Colesterol_LDL'], row['count_topic_Día_de_la_independencia'], row['count_topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva'], row['count_topic_Día_Mundial_de_la_Leucemia'], row['count_topic_Día_Mundial_de_la_seguridad_del_paciente'], row['count_topic_Día_Mundial_de_la_Trombocitopenia_Inmune'], row['count_topic_Día_Mundial_del_Corazón'], row['count_topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos'], row['count_topic_Estudio_enfermedades_cardiovasculares'], row['count_topic_GPTW'], row['count_topic_Hematopedia'], row['count_topic_LDL_Xchange'], row['count_topic_LLA'], row['count_topic_Mieloma_Múltiple'], row['count_topic_Reporte_ESG'], row['count_topic_Salud_ósea'], row['count_topic_We_Are_Amgen']), axis=1)
print(amgen_fb_df.shape)
amgen_fb_df.head()

(39, 72)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,count_topic_LLA,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,0,[],0,[],0,[],0,[WeAreAmgen],1,GPTW
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,0,[],0,[],0,[],0,[],0,LDL Xchange
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,0,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Día Mundial de la Leucemia
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,0,[],0,[],0,[],0,[],0,Día Mundial de la Leucemia


In [657]:
amgen_fb_df['Tema_pred']

0                                                  GPTW
1                                           LDL Xchange
2                            Día Mundial de la Leucemia
3                                     Alerta Colesterol
4                            Día Mundial de la Leucemia
5                                     Alerta Colesterol
6     Día Internacional de las personas con discapac...
7                                     Alerta Colesterol
8     Día Nacional de la donación y trasplante de ór...
9                                     Alerta Colesterol
10                                    Alerta Colesterol
11             Día Mundial de la Trombocitopenia Inmune
12                                          LDL Xchange
13                                    Alerta Colesterol
14                                    Alerta Colesterol
15                           Día Mundial de la Leucemia
16                           Día Mundial de la Leucemia
17                           Día Mundial de la L

## Select Final Columns

In [658]:
print(amgen_fb_df.columns)

Index(['Identificador de la publicación', 'Identificador de la página',
       'Nombre de la página', 'Título', 'Descripción', 'Hora de publicación',
       'Enlace permanente', 'Personas alcanzadas', 'Impresiones',
       'Veces que se compartió', 'Me gusta', 'Comentarios', 'Total de clics',
       'Reproducciones de video de 3 segundos', 'Interacciones', 'Categoría',
       'Tema', 'Formato', 'Día', 'Hora', 'cat_Biocomparables',
       'count_cat_Biocomparables', 'cat_Cardiovascular',
       'count_cat_Cardiovascular', 'cat_Corporativo', 'count_cat_Corporativo',
       'cat_Hematología', 'count_cat_Hematología', 'cat_Osteoporosis',
       'count_cat_Osteoporosis', 'Categoría_pred', 'topic_Alerta_Colesterol',
       'count_topic_Alerta_Colesterol', 'topic_Biocomparables',
       'count_topic_Biocomparables', 'topic_Cánceres_hematológicos',
       'count_topic_Cánceres_hematológicos', 'topic_Colesterol_LDL',
       'count_topic_Colesterol_LDL', 'topic_Día_de_la_independencia',
       '

In [659]:
amgen_fb_df['ER'] = (amgen_fb_df['Interacciones']/amgen_fb_df['Impresiones'])*100
amgen_fb_df['ER'] = amgen_fb_df['ER'].round(2).astype(str) + '%'
print(amgen_fb_df.shape)
amgen_fb_df.head(3)

(39, 73)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred,ER
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,[],0,[],0,[],0,[WeAreAmgen],1,GPTW,3.39%
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,[],0,[],0,[],0,[],0,LDL Xchange,4.88%
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Día Mundial de la Leucemia,6.12%


In [660]:
amgen_fb_df_final = amgen_fb_df[[
'Identificador de la publicación',
'Identificador de la página',
'Nombre de la página',
'Título',
'Descripción',
'Hora de publicación',
'Enlace permanente',
'Categoría_pred',
'Tema_pred',
'Formato',
'Día',
'Hora',
'Personas alcanzadas',
'Impresiones',
'Interacciones',
'ER',
'Veces que se compartió',
'Me gusta',
'Comentarios',
'Total de clics',
'Reproducciones de video de 3 segundos']]
print(amgen_fb_df_final.shape)
amgen_fb_df_final.head()

(39, 21)


,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Categoría_pred,Tema_pred,Formato,...,Hora,Personas alcanzadas,Impresiones,Interacciones,ER,Veces que se compartió,Me gusta,Comentarios,Total de clics,Reproducciones de video de 3 segundos
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,Corporativo,GPTW,,...,,470,472,16,3.39%,0,13,0,3,0
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,Cardiovascular,LDL Xchange,,...,,116,123,6,4.88%,0,5,0,1,0
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,Hematología,Día Mundial de la Leucemia,,...,,49,49,3,6.12%,0,3,0,0,0
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,Cardiovascular,Alerta Colesterol,,...,,42,44,4,9.09%,0,4,0,0,16
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,Hematología,Día Mundial de la Leucemia,,...,,117,120,9,7.5%,1,6,0,2,0


## Dataframes To Add - Excel Master File (Facebook)

In [618]:
amgen_fb_df_final # Procesado con Tema y Categoría Sugerido Según Keywords y Pilares
amgen_fb_df # Versión con Todas las Columnas de Categorías-Keywords & Temas-Keywords 

,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred,ER
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,[],0,[],0,[],0,[WeAreAmgen],1,GPTW,3.39%
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,[],0,[],0,[],0,[],0,LDL Xchange,4.88%
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Hematopedia,6.12%
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,[],0,[],0,[],0,[],0,Alerta Colesterol,9.09%
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,[],0,[],0,[],0,[],0,Día Mundial de la Trombocitopenia Inmune,7.5%
5,813945380730699,706715436424096,Amgen México,Enfermedades del corazón,Las enfermedades del corazón y los accidentes ...,09/27/2023 16:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,21655,23905,2,...,[],0,[],0,[],0,[],0,Alerta Colesterol,1.84%
6,813734477418456,706715436424096,Amgen México,NaN,"En México, aproximadamente 2.3 millones de per...",09/27/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0q...,70,72,0,...,[],0,[],0,[],0,[WeAreAmgen],1,None,4.17%
7,813262300799007,706715436424096,Amgen México,NaN,Reducir los niveles de colesterol malo (LDL) e...,09/26/2023 16:09,https://www.facebook.com/AmgenMx/posts/pfbid0S...,11460,14779,49,...,[],0,[],0,[],0,[],0,Alerta Colesterol,7.15%
8,813029414155629,706715436424096,Amgen México,NaN,"Para muchas personas, un trasplante es la únic...",09/26/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0P...,87,90,0,...,[],0,[],0,[],0,[WeAreAmgen],1,None,3.33%
9,812562824202288,706715436424096,Amgen México,Colesterol LDL,El colesterol LDL o “malo” puede causar daños ...,09/25/2023 15:09,https://www.facebook.com/AmgenMx/posts/pfbid0o...,16796,22323,1,...,[],0,[],0,[],0,[],0,Alerta Colesterol,0.63%


### LinkedIn

In [661]:
amgen_linkedin_df = pd.read_excel('source_data_raw/september/linkedin/amgen_content_1696261306209.xls', sheet_name= 'Todas las publicaciones', skiprows=1)
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(138, 20)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Nombre de la campaña,Anunciado por,Fecha de creación,Fecha de inicio de campaña,Fecha de finalización de campaña,Público,Impresiones,Visualizaciones (excluyendo las visualizaciones de vídeos fuera del sitio),Visualizaciones fuera del sitio,Clics,Porcentaje de clics,Recomendaciones,Comentarios,Veces compartido,Seguidores,Tasa de interacción,Tipo de contenido
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,NaN,Antonio Guerrero Sánchez,08/29/2023,NaN,NaN,Segmentada,163,NaN,NaN,3,0.018405,5,0,1,NaN,0.055215,NaN
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,NaN,Antonio Guerrero Sánchez,09/18/2023,NaN,NaN,Segmentada,144,NaN,NaN,0,0.000000,2,0,1,NaN,0.020833,NaN
2,There are many ways to describe Arleen Paulino...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,NaN,Annie Sullivan,09/30/2023,NaN,NaN,Todos los seguidores,16884,6799.0,NaN,309,0.018301,322,12,3,NaN,0.038380,Vídeo


In [662]:
amgen_linkedin_df = amgen_linkedin_df[['Título de la publicación',
                              'Enlace de la publicación',
                              'Tipo de publicación',
                              'Anunciado por',
                              'Fecha de creación',
                              'Público',
                              'Impresiones',
                              'Recomendaciones',
                              'Comentarios',
                              'Veces compartido',
                              'Clics',
                              'Visualizaciones (excluyendo las visualizaciones de vídeos fuera del sitio)'
                              ]]
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(138, 12)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones (excluyendo las visualizaciones de vídeos fuera del sitio)
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,NaN
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,NaN
2,There are many ways to describe Arleen Paulino...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Annie Sullivan,09/30/2023,Todos los seguidores,16884,322,12,3,309,6799.0


In [663]:
amgen_linkedin_df.rename(columns={'Visualizaciones (excluyendo las visualizaciones de vídeos fuera del sitio)':'Visualizaciones'}, inplace=True)
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head()

(138, 12)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,NaN
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,NaN
2,There are many ways to describe Arleen Paulino...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Annie Sullivan,09/30/2023,Todos los seguidores,16884,322,12,3,309,6799.0
3,As we conclude Immune Thrombocytopenia (ITP) a...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Andy Substance,09/30/2023,Segmentada,364,11,0,0,5,131.0
4,Hoje comemora-se o Dia do Profissional de Secr...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Paola Andrade,09/30/2023,Segmentada,1129,35,1,1,65,NaN


In [664]:
amgen_linkedin_df['Impresiones'] = amgen_linkedin_df['Impresiones'].astype('int64')
amgen_linkedin_df['Recomendaciones'] = amgen_linkedin_df['Recomendaciones'].astype('int64')
amgen_linkedin_df['Comentarios'] = amgen_linkedin_df['Comentarios'].astype('int64')
amgen_linkedin_df['Veces compartido'] = amgen_linkedin_df['Veces compartido'].astype('int64')
amgen_linkedin_df['Clics'] = amgen_linkedin_df['Clics'].astype('int64')
amgen_linkedin_df['Visualizaciones'] = amgen_linkedin_df['Visualizaciones'].fillna(0)
amgen_linkedin_df['Visualizaciones'] = amgen_linkedin_df['Visualizaciones'].astype('int64')
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(138, 12)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,0
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,0
2,There are many ways to describe Arleen Paulino...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Annie Sullivan,09/30/2023,Todos los seguidores,16884,322,12,3,309,6799


In [665]:
amgen_linkedin_df = amgen_linkedin_df[amgen_linkedin_df['Anunciado por'].isin(['Antonio Guerrero Sánchez', 'Rodolfo Zapata Aguilar', 'Suraya'])]
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(35, 12)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,0
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,0
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,5,178


In [666]:
amgen_linkedin_df['Interacciones']= amgen_linkedin_df['Recomendaciones'] + amgen_linkedin_df['Comentarios'] + amgen_linkedin_df['Veces compartido'] + amgen_linkedin_df['Clics'] + amgen_linkedin_df['Visualizaciones']
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(35, 13)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones,Interacciones
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,0,9
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,0,3
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,5,178,204


In [668]:
amgen_linkedin_df['ER'] = (amgen_linkedin_df['Interacciones']/amgen_linkedin_df['Impresiones'])*100
amgen_linkedin_df['ER'] = amgen_linkedin_df['ER'].round(2).astype(str) + '%'
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(35, 14)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones,Interacciones,ER
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,0,9,5.52%
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,0,3,2.08%
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,5,178,204,76.12%


In [669]:
amgen_linkedin_df['Categoría'] = ''
amgen_linkedin_df['Tema'] = ''
amgen_linkedin_df['Formato'] = ''
amgen_linkedin_df['Día'] = ''
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head(3)

(35, 18)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones,Interacciones,ER,Categoría,Tema,Formato,Día
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,3,0,9,5.52%,,,,
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,0,0,3,2.08%,,,,
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,5,178,204,76.12%,,,,


## Categories

In [670]:
for k,v in categories_dicc.items():
    amgen_linkedin_df[f'{k}']= amgen_linkedin_df['Título de la publicación'].apply(lambda x: palabras_presentes(x, v))
    # in orher column count the finded words
    amgen_linkedin_df[f'count_{k}'] = amgen_linkedin_df['Título de la publicación'].apply(lambda x: contar_palabras(x, v))
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head()

(35, 28)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,...,cat_Biocomparables,count_cat_Biocomparables,cat_Cardiovascular,count_cat_Cardiovascular,cat_Corporativo,count_cat_Corporativo,cat_Hematología,count_cat_Hematología,cat_Osteoporosis,count_cat_Osteoporosis
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,...,[],0,"[salud cardiovascular, enfermedad ateroesclero...",4,[],0,[],0,[],0
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,...,[],0,[Organización Mundial de la Salud],1,[],0,"[Mieloma Múltiple, neoplasias hematológicas, p...",11,[],0
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,...,[],0,"[colesterol, cardiovasculares, colesterol LDL,...",15,[Amgen],1,[],0,[],0
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,1241,10,0,1,...,[],0,[],0,[],0,"[plaquetas, TIP, trombocitopenia, Día Mundial ...",6,[],0
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,2920,36,0,2,...,[],0,"[colesterol, cardiovasculares, colesterol malo...",9,[],0,[],0,[],0


In [671]:
amgen_linkedin_df['Categoría_pred'] = amgen_linkedin_df.apply(lambda row: assign_categoria(row['count_cat_Cardiovascular'], row['count_cat_Hematología'], row['count_cat_Osteoporosis'], row['count_cat_Biocomparables'], row['count_cat_Corporativo']), axis=1)
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head()

(35, 29)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,...,count_cat_Biocomparables,cat_Cardiovascular,count_cat_Cardiovascular,cat_Corporativo,count_cat_Corporativo,cat_Hematología,count_cat_Hematología,cat_Osteoporosis,count_cat_Osteoporosis,Categoría_pred
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,...,0,"[salud cardiovascular, enfermedad ateroesclero...",4,[],0,[],0,[],0,Cardiovascular
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,...,0,[Organización Mundial de la Salud],1,[],0,"[Mieloma Múltiple, neoplasias hematológicas, p...",11,[],0,Hematología
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,...,0,"[colesterol, cardiovasculares, colesterol LDL,...",15,[Amgen],1,[],0,[],0,Cardiovascular
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,1241,10,0,1,...,0,[],0,[],0,"[plaquetas, TIP, trombocitopenia, Día Mundial ...",6,[],0,Hematología
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,2920,36,0,2,...,0,"[colesterol, cardiovasculares, colesterol malo...",9,[],0,[],0,[],0,Cardiovascular


## Topics

In [672]:
for k,v in topics_dicc.items():
    amgen_linkedin_df[f'{k}']= amgen_linkedin_df['Título de la publicación'].apply(lambda x: palabras_presentes(x, v))
    # in orher column count the finded words
    amgen_linkedin_df[f'count_{k}'] = amgen_linkedin_df['Título de la publicación'].apply(lambda x: contar_palabras(x, v))
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head()

(35, 69)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,...,topic_LLA,count_topic_LLA,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,...,[],0,[],0,[],0,[],0,[],0
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,...,[],0,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,...,[],0,[],0,[],0,[],0,[],0
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,1241,10,0,1,...,[],0,[],0,[],0,[],0,[],0
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,2920,36,0,2,...,[],0,[],0,[],0,[],0,[],0


In [673]:
amgen_linkedin_df['Tema_pred'] = amgen_linkedin_df.apply(lambda row: assign_topico(row['count_topic_Alerta_Colesterol'], row['count_topic_Biocomparables'], row['count_topic_Cánceres_hematológicos'], row['count_topic_Colesterol_LDL'], row['count_topic_Día_de_la_independencia'], row['count_topic_Día_Internacional_de_las_personas_con_discapacidad_auditiva'], row['count_topic_Día_Mundial_de_la_Leucemia'], row['count_topic_Día_Mundial_de_la_seguridad_del_paciente'], row['count_topic_Día_Mundial_de_la_Trombocitopenia_Inmune'], row['count_topic_Día_Mundial_del_Corazón'], row['count_topic_Día_Nacional_de_la_donación_y_trasplante_de_órganos'], row['count_topic_Estudio_enfermedades_cardiovasculares'], row['count_topic_GPTW'], row['count_topic_Hematopedia'], row['count_topic_LDL_Xchange'], row['count_topic_LLA'], row['count_topic_Mieloma_Múltiple'], row['count_topic_Reporte_ESG'], row['count_topic_Salud_ósea'], row['count_topic_We_Are_Amgen']), axis=1)
print(amgen_linkedin_df.shape)
amgen_linkedin_df.head()

(35, 70)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,...,count_topic_LLA,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,...,0,[],0,[],0,[],0,[],0,LDL Xchange
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,...,0,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Día Mundial de la Leucemia
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,1241,10,0,1,...,0,[],0,[],0,[],0,[],0,Día Mundial de la Leucemia
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,2920,36,0,2,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol


In [674]:
amgen_linkedin_df_final = amgen_linkedin_df[[
'Título de la publicación',
'Enlace de la publicación',
'Tipo de publicación',
'Anunciado por',
'Fecha de creación',
'Público',
'Categoría_pred',
'Tema_pred',
'Formato',
'Día',
'Impresiones',
'Interacciones',
'ER',
'Recomendaciones',
'Comentarios',
'Veces compartido',
'Clics',
'Visualizaciones'
]]
print(amgen_linkedin_df_final.shape)
amgen_linkedin_df_final.head()

(35, 18)


,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Categoría_pred,Tema_pred,Formato,Día,Impresiones,Interacciones,ER,Recomendaciones,Comentarios,Veces compartido,Clics,Visualizaciones
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,Cardiovascular,LDL Xchange,,,163,9,5.52%,5,0,1,3,0
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,Hematología,Día Mundial de la Leucemia,,,144,3,2.08%,2,0,1,0,0
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,Cardiovascular,Alerta Colesterol,,,268,204,76.12%,16,0,5,5,178
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,Hematología,Día Mundial de la Leucemia,,,1241,19,1.53%,10,0,1,8,0
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,Cardiovascular,Alerta Colesterol,,,2920,843,28.87%,36,0,2,34,771


## Dataframes To Add - Excel Master File (LinkedIn)

In [675]:
amgen_linkedin_df_final # Procesado con Tema y Categoría Sugerido Según Keywords y Pilares
amgen_linkedin_df # Versión con Todas las Columnas de Categorías-Keywords & Temas-Keywords 

,Título de la publicación,Enlace de la publicación,Tipo de publicación,Anunciado por,Fecha de creación,Público,Impresiones,Recomendaciones,Comentarios,Veces compartido,...,count_topic_LLA,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred
0,"En el último módulo de #LDLXChange, las y los ...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/29/2023,Segmentada,163,5,0,1,...,0,[],0,[],0,[],0,[],0,LDL Xchange
1,La Organización Mundial de la Salud (OMS) indi...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,144,2,0,1,...,0,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Día Mundial de la Leucemia
5,"En el marco del #DíaMundialdelCorazón, el equi...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/28/2023,Segmentada,268,16,0,5,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol
6,Hoy se conmemora el Día Mundial de la Tromboci...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,1241,10,0,1,...,0,[],0,[],0,[],0,[],0,Día Mundial de la Leucemia
8,¡Es oficialmente el #DíaMundialdelCorazón! \n\...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,2920,36,0,2,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol
21,Las células sanguíneas llamadas plaquetas son ...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/18/2023,Segmentada,169,9,0,0,...,0,[],0,[],0,[],0,[],0,Día Mundial de la Trombocitopenia Inmune
23,El 29 de septiembre conmemoramos el Día Mundia...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Rodolfo Zapata Aguilar,09/28/2023,Segmentada,374,22,0,0,...,0,[],0,[],0,[],0,[],0,Día Mundial de la Leucemia
24,Reducir los niveles de colesterol malo (LDL) e...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,154,7,0,1,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol
27,"En México, aproximadamente 2.3 millones de per...",https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,08/23/2023,Segmentada,238,9,0,1,...,0,[],0,[],0,[],0,[WeAreAmgen],1,Día Internacional de las personas con discapac...
30,El colesterol LDL o “malo” puede causar daños ...,https://www.linkedin.com/feed/update/urn:li:ac...,Orgánico,Antonio Guerrero Sánchez,09/12/2023,Segmentada,318,10,0,0,...,0,[],0,[],0,[],0,[],0,Alerta Colesterol


In [676]:
amgen_fb_df.head()

,Identificador de la publicación,Identificador de la página,Nombre de la página,Título,Descripción,Hora de publicación,Enlace permanente,Personas alcanzadas,Impresiones,Veces que se compartió,...,topic_Mieloma_Múltiple,count_topic_Mieloma_Múltiple,topic_Reporte_ESG,count_topic_Reporte_ESG,topic_Salud_ósea,count_topic_Salud_ósea,topic_We_Are_Amgen,count_topic_We_Are_Amgen,Tema_pred,ER
0,796765385782032,706715436424096,Amgen México,NaN,En Amgen estamos sumamente orgullosos de conti...,09/02/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,470,472,0,...,[],0,[],0,[],0,[WeAreAmgen],1,GPTW,3.39%
1,796022045856366,706715436424096,Amgen México,NaN,Para Amgen la educación médica continua es mue...,09/01/2023 08:09,https://www.facebook.com/AmgenMx/posts/pfbid0B...,116,123,0,...,[],0,[],0,[],0,[],0,LDL Xchange,4.88%
2,815765847215319,706715436424096,Amgen México,NaN,La Organización Mundial de la Salud (OMS) indi...,09/30/2023 09:09,https://www.facebook.com/AmgenMx/posts/pfbid0A...,49,49,0,...,"[mieloma múltiple, Mieloma Múltiple]",6,[],0,[],0,[],0,Día Mundial de la Leucemia,6.12%
3,815225267269377,706715436424096,Amgen México,Día Mundial del Corazón,¡Es oficialmente el #DíaMundialdelCorazón!\n \...,09/29/2023 12:09,https://www.facebook.com/AmgenMx/posts/pfbid0p...,42,44,0,...,[],0,[],0,[],0,[],0,Alerta Colesterol,9.09%
4,815085517283352,706715436424096,Amgen México,NaN,Hoy se conmemora el Día Mundial de la Tromboci...,09/29/2023 07:09,https://www.facebook.com/AmgenMx/posts/pfbid02...,117,120,1,...,[],0,[],0,[],0,[],0,Día Mundial de la Leucemia,7.5%


In [677]:
import pandas as pd

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Amgen September Report.xlsx', engine='xlsxwriter')

# Write each DataFrame to a different worksheet.
amgen_fb_df.to_excel(writer, sheet_name='Amgen Facebook Preliminar', index=False)
amgen_fb_df_final.to_excel(writer, sheet_name='Amgen Facebook Final', index=False)
amgen_linkedin_df.to_excel(writer, sheet_name='Amgen LinkedIn Preliminar', index=False)
amgen_linkedin_df_final.to_excel(writer, sheet_name='Amgen LinkedIn Final', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()


### Twitter

In [233]:
amgen_twitter_df = pd.read_csv('source_data_raw/september/twitter/tweet_activity_metrics_AmgenMexico_20230901_20231001_en.csv')
print(amgen_twitter_df.shape)
amgen_twitter_df.head(3)

(38, 40)


,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37.0,1.0,0.027027,0.0,0.0,1.0,...,-,-,-,-,-,-,-,-,-,-
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57.0,3.0,0.052632,0.0,0.0,3.0,...,-,-,-,-,-,-,-,-,-,-
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43.0,1.0,0.023256,0.0,0.0,1.0,...,-,-,-,-,-,-,-,-,-,-


In [234]:
amgen_twitter_df = amgen_twitter_df[['Tweet id',
                              'Tweet permalink',
                              'Tweet text',
                              'time',
                              'impressions',
                              'retweets',
                              'replies',
                              'likes',
                              #'clics',
                              'media views',
                              'promoted impressions',
                              'promoted retweets',
                              'promoted replies',
                              'promoted likes',
                              'promoted media views',
                              'promoted user profile clicks',
                              'promoted url clicks',
                              'promoted hashtag clicks',
                              'promoted detail expands',
                              'promoted follows',
                              'promoted engagements'
                              ]]
print(amgen_twitter_df.shape)
amgen_twitter_df.head(3)

(38, 20)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,promoted retweets,promoted replies,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37.0,0.0,0.0,1.0,0,-,-,-,-,-,-,-,-,-,-,-
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57.0,0.0,0.0,3.0,12,-,-,-,-,-,-,-,-,-,-,-
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43.0,0.0,0.0,1.0,0,-,-,-,-,-,-,-,-,-,-,-


In [235]:
#amgen_twitter_df = amgen_twitter_df.astype(str)

In [236]:
#amgen_twitter_df['impressions'] = amgen_twitter_df['impressions'].fillna(0)
#amgen_twitter_df['retweets'] = amgen_twitter_df['retweets'].fillna(0)
#amgen_twitter_df['replies'] = amgen_twitter_df['replies'].fillna(0)
#amgen_twitter_df['likes'] = amgen_twitter_df['likes'].fillna(0)
#amgen_twitter_df['media views'] = amgen_twitter_df['media views'].fillna(0)
#amgen_twitter_df['promoted impressions'] = amgen_twitter_df['promoted impressions'].fillna(0)
#amgen_twitter_df['promoted likes'] = amgen_twitter_df['promoted likes'].fillna(0)
#amgen_twitter_df['promoted media views'] = amgen_twitter_df['promoted media views'].fillna(0)
#amgen_twitter_df['promoted user profile clicks'] = amgen_twitter_df['promoted user profile clicks'].fillna(0)
#amgen_twitter_df['promoted url clicks'] = amgen_twitter_df['promoted url clicks'].fillna(0)
#amgen_twitter_df['promoted hashtag clicks'] = amgen_twitter_df['promoted hashtag clicks'].fillna(0)
#amgen_twitter_df['promoted detail expands'] = amgen_twitter_df['promoted detail expands'].fillna(0)
#amgen_twitter_df['promoted follows'] = amgen_twitter_df['promoted follows'].fillna(0)
#print(amgen_twitter_df.shape)
#amgen_twitter_df.head()

In [237]:
amgen_twitter_df['impressions'] = pd.to_numeric(amgen_twitter_df['impressions'], downcast='integer')
amgen_twitter_df['retweets'] = pd.to_numeric(amgen_twitter_df['retweets'], downcast='integer')
amgen_twitter_df['replies'] = pd.to_numeric(amgen_twitter_df['replies'], downcast='integer')
amgen_twitter_df['likes'] = pd.to_numeric(amgen_twitter_df['likes'], downcast='integer')

amgen_twitter_df['media views'] = pd.to_numeric(amgen_twitter_df['media views'], downcast='integer')
amgen_twitter_df['promoted impressions'] = pd.to_numeric(amgen_twitter_df['promoted impressions'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted likes'] = pd.to_numeric(amgen_twitter_df['promoted likes'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted media views'] = pd.to_numeric(amgen_twitter_df['promoted media views'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted user profile clicks'] = pd.to_numeric(amgen_twitter_df['promoted user profile clicks'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted url clicks'] = pd.to_numeric(amgen_twitter_df['promoted url clicks'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted hashtag clicks'] = pd.to_numeric(amgen_twitter_df['promoted hashtag clicks'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted detail expands'] = pd.to_numeric(amgen_twitter_df['promoted detail expands'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted follows'] = pd.to_numeric(amgen_twitter_df['promoted follows'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted engagements'] = pd.to_numeric(amgen_twitter_df['promoted engagements'], downcast='integer', errors='coerce')
amgen_twitter_df['promoted replies'] = pd.to_numeric(amgen_twitter_df['promoted replies'], downcast='integer', errors='coerce')


print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 20)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,promoted retweets,promoted replies,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988.0,11.0,1.0,96.0,2381.0,6.0,0.0,5.0,48.0,0.0,189.0
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
amgen_twitter_df['impressions'] = amgen_twitter_df['impressions'].fillna(0)
amgen_twitter_df['retweets'] = amgen_twitter_df['retweets'].fillna(0)
amgen_twitter_df['replies'] = amgen_twitter_df['replies'].fillna(0)
amgen_twitter_df['likes'] = amgen_twitter_df['likes'].fillna(0)
amgen_twitter_df['media views'] = amgen_twitter_df['media views'].fillna(0)
amgen_twitter_df['promoted impressions'] = amgen_twitter_df['promoted impressions'].fillna(0)
amgen_twitter_df['promoted likes'] = amgen_twitter_df['promoted likes'].fillna(0)
amgen_twitter_df['promoted media views'] = amgen_twitter_df['promoted media views'].fillna(0)
amgen_twitter_df['promoted user profile clicks'] = amgen_twitter_df['promoted user profile clicks'].fillna(0)
amgen_twitter_df['promoted url clicks'] = amgen_twitter_df['promoted url clicks'].fillna(0)
amgen_twitter_df['promoted hashtag clicks'] = amgen_twitter_df['promoted hashtag clicks'].fillna(0)
amgen_twitter_df['promoted detail expands'] = amgen_twitter_df['promoted detail expands'].fillna(0)
amgen_twitter_df['promoted follows'] = amgen_twitter_df['promoted follows'].fillna(0)
amgen_twitter_df['promoted engagements'] = amgen_twitter_df['promoted engagements'].fillna(0)
amgen_twitter_df['promoted replies'] = amgen_twitter_df['promoted replies'].fillna(0)


print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 20)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,promoted retweets,promoted replies,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0.0,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0.0,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0.0,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988.0,11.0,1.0,96.0,2381.0,6.0,0.0,5.0,48.0,0.0,189.0
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0.0,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [239]:
amgen_twitter_df['impressions'] = amgen_twitter_df['impressions'].astype('int')
amgen_twitter_df['retweets'] = amgen_twitter_df['retweets'].astype('int')
amgen_twitter_df['replies'] = amgen_twitter_df['replies'].astype('int')
amgen_twitter_df['likes'] = amgen_twitter_df['likes'].astype('int')
amgen_twitter_df['media views'] = amgen_twitter_df['media views'].astype('int')
amgen_twitter_df['promoted impressions'] = amgen_twitter_df['promoted impressions'].astype('int')
amgen_twitter_df['promoted likes'] = amgen_twitter_df['promoted likes'].astype('int')
amgen_twitter_df['promoted media views'] = amgen_twitter_df['promoted media views'].astype('int')
amgen_twitter_df['promoted user profile clicks'] = amgen_twitter_df['promoted user profile clicks'].astype('int')
amgen_twitter_df['promoted url clicks'] = amgen_twitter_df['promoted url clicks'].astype('int')
amgen_twitter_df['promoted hashtag clicks'] = amgen_twitter_df['promoted hashtag clicks'].astype('int')
amgen_twitter_df['promoted detail expands'] = amgen_twitter_df['promoted detail expands'].astype('int')
amgen_twitter_df['promoted follows'] = amgen_twitter_df['promoted follows'].astype('int')
amgen_twitter_df['promoted engagements'] = amgen_twitter_df['promoted engagements'].astype('int')
amgen_twitter_df['promoted replies'] = amgen_twitter_df['promoted replies'].astype('int')


print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 20)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,promoted retweets,promoted replies,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,-,0,0,0,0,0,0,0,0,0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,-,0,0,0,0,0,0,0,0,0
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,-,0,0,0,0,0,0,0,0,0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,11.0,1,96,2381,6,0,5,48,0,189
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,-,0,0,0,0,0,0,0,0,0


In [240]:
amgen_twitter_df['interacciones'] = amgen_twitter_df['retweets'] + amgen_twitter_df['replies'] + amgen_twitter_df['likes'] + amgen_twitter_df['retweets'] + amgen_twitter_df['media views']
print(amgen_twitter_df.shape)
amgen_twitter_df.head(3)

(38, 21)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted replies,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,0,0,0,0,0,15
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [241]:
amgen_twitter_df['ER'] = (amgen_twitter_df['interacciones']/amgen_twitter_df['impressions'])*100
amgen_twitter_df['ER'] = amgen_twitter_df['ER'].round(2)
print(amgen_twitter_df.shape)
amgen_twitter_df.head(3)

(38, 22)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted likes,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones,ER
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2.70
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,0,0,0,0,15,26.32
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2.33


In [242]:
amgen_twitter_df['promoted clics'] = amgen_twitter_df['promoted user profile clicks'] + amgen_twitter_df['promoted url clicks'] + amgen_twitter_df['promoted hashtag clicks'] + amgen_twitter_df['promoted detail expands'] + amgen_twitter_df['promoted follows']
print(amgen_twitter_df.shape)
amgen_twitter_df.head(5)

(38, 23)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted media views,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones,ER,promoted clics
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,0,0,0,1,2.70,0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,0,0,0,15,26.32,0
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,0,0,0,1,2.33,0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,2381,6,0,5,48,0,189,57,22.35,59
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,0,0,0,0,0,0,4,5.97,0


In [243]:
amgen_twitter_df['IMPRESSIONS'] = amgen_twitter_df['impressions'] + amgen_twitter_df['promoted impressions']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 24)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted user profile clicks,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,0,0,1,2.70,0,37
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,0,0,15,26.32,0,57
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,0,0,1,2.33,0,43
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,6,0,5,48,0,189,57,22.35,59,11243
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,0,0,0,0,0,4,5.97,0,67


In [244]:
amgen_twitter_df['INTERACTIONS'] = amgen_twitter_df['interacciones'] + amgen_twitter_df['promoted engagements']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 25)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted url clicks,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,0,1,2.70,0,37,1
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,0,15,26.32,0,57,15
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,0,1,2.33,0,43,1
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,0,5,48,0,189,57,22.35,59,11243,246
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,0,0,0,0,4,5.97,0,67,4


In [245]:
amgen_twitter_df['ER_NEW'] = (amgen_twitter_df['INTERACTIONS']/amgen_twitter_df['IMPRESSIONS'])*100
amgen_twitter_df['ER_NEW'] = amgen_twitter_df['ER_NEW'].round(2)
print(amgen_twitter_df.shape)
amgen_twitter_df.head(3)

(38, 26)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted hashtag clicks,promoted detail expands,promoted follows,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS,ER_NEW
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,0,1,2.70,0,37,1,2.70
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,0,15,26.32,0,57,15,26.32
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,0,1,2.33,0,43,1,2.33


In [246]:
amgen_twitter_df['REPLIES'] = amgen_twitter_df['replies'] + amgen_twitter_df['promoted replies']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 27)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted detail expands,promoted follows,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS,ER_NEW,REPLIES
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,0,1,2.70,0,37,1,2.70,0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,0,15,26.32,0,57,15,26.32,0
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,0,1,2.33,0,43,1,2.33,0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,48,0,189,57,22.35,59,11243,246,2.19,1
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,0,0,4,5.97,0,67,4,5.97,0


In [247]:
amgen_twitter_df['LIKES'] = amgen_twitter_df['likes'] + amgen_twitter_df['promoted likes']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 28)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted follows,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS,ER_NEW,REPLIES,LIKES
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,0,1,2.70,0,37,1,2.70,0,1
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,0,15,26.32,0,57,15,26.32,0,3
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,0,1,2.33,0,43,1,2.33,0,1
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,0,189,57,22.35,59,11243,246,2.19,1,96
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,0,4,5.97,0,67,4,5.97,0,3


In [248]:
amgen_twitter_df['CLICS'] = amgen_twitter_df['promoted clics']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()
# + amgen_twitter_df['clics']

(38, 29)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,promoted engagements,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS,ER_NEW,REPLIES,LIKES,CLICS
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,0,1,2.70,0,37,1,2.70,0,1,0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,0,15,26.32,0,57,15,26.32,0,3,0
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,0,1,2.33,0,43,1,2.33,0,1,0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,189,57,22.35,59,11243,246,2.19,1,96,59
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,0,4,5.97,0,67,4,5.97,0,3,0


In [249]:
amgen_twitter_df['VIEWS'] = amgen_twitter_df['media views'] + amgen_twitter_df['promoted media views']
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 30)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,interacciones,ER,promoted clics,IMPRESSIONS,INTERACTIONS,ER_NEW,REPLIES,LIKES,CLICS,VIEWS
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,1,2.70,0,37,1,2.70,0,1,0,0
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,15,26.32,0,57,15,26.32,0,3,0,12
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,1,2.33,0,43,1,2.33,0,1,0,0
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,57,22.35,59,11243,246,2.19,1,96,59,2434
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,4,5.97,0,67,4,5.97,0,3,0,1


In [250]:
amgen_twitter_df['Categoría']= ''
amgen_twitter_df['Tema']= ''
amgen_twitter_df['Formato']= ''
amgen_twitter_df['Día']= ''
amgen_twitter_df['Hora']= ''
print(amgen_twitter_df.shape)
amgen_twitter_df.head()

(38, 35)


,Tweet id,Tweet permalink,Tweet text,time,impressions,retweets,replies,likes,media views,promoted impressions,...,ER_NEW,REPLIES,LIKES,CLICS,VIEWS,Categoría,Tema,Formato,Día,Hora
0,1708149711645892871,https://twitter.com/AmgenMexico/status/1708149...,La OMS indica que el mieloma múltiple represen...,2023-09-30 16:00 +0000,37,0,0,1,0,0,...,2.70,0,1,0,0,,,,,
1,1707832648641532096,https://twitter.com/AmgenMexico/status/1707832...,¡Es oficialmente el #DíaMundialdelCorazón! Las...,2023-09-29 19:00 +0000,57,0,0,3,12,0,...,26.32,0,3,0,12,,,,,
2,1707764672290500646,https://twitter.com/AmgenMexico/status/1707764...,Hoy se conmemora el Día Mundial de la Tromboci...,2023-09-29 14:30 +0000,43,0,0,1,0,0,...,2.33,0,1,0,0,,,,,
3,1707168267188473971,https://twitter.com/AmgenMexico/status/1707168...,Las enfermedades del corazón y accidentes cere...,2023-09-27 23:00 +0000,255,2,0,0,53,10988,...,2.19,1,96,59,2434,,,,,
4,1707056263509520824,https://twitter.com/AmgenMexico/status/1707056...,En @Amgen estamos comprometidos con el talento...,2023-09-27 15:35 +0000,67,0,0,3,1,0,...,5.97,0,3,0,1,,,,,
